In [758]:
import xpress as xp
import numpy as np

原文中可以改进的地方：
1. A.6没有考虑前文提到的，因为独立日比赛，最强两队的要求应当减少
2. A.22中的Z没有标明
3. A.25和A.26可以合并到A.22中
4. A.29公式没有写全ijk+jik
5. A.34公式没有补充其他球队独立日的情况

在我们整理了2020年前后几年的夏季赛数据后发现，7月4日有且必定仅有固定的两支队伍比赛，我们思考后意识到，
首先，7月4日是美国的独立日，主办方会为该日期安排特殊赛程，
其次，固定在独立日比赛的两支队伍综合实力是最强的，意味他们两队的比赛会有很多观众，收视率很高
并且，其余队伍不会安排任何比赛，我们认为也是为了让球员也能庆祝独立日

此外，由于7月4日是星期六，而主办方通常安排周日满赛，周一公休，
如果，参考过往全明星赛的规律，将一定排在周日的全明星赛安排在7月5日，会有连续三天休息：独立日+全明星+公休
所以，应避免将全明星赛安排在7月4日所在的第六周，可选的就是前后某一周，
又因为论文中提到，第五周第六周的休赛日多，所以可推测全明星日安排在第五周的周日。

In [759]:
WEEKENDS = [0,1, 7,8, 14,15, 21,22, 28,29, 35, 42,43, 49,50]
# WEEKENDS = [0,1, 7,8, 14,15, 21,22, 28,29, 35,36, 42,43, 49,50]
is_rep = False
is_line = False
YEAR = 2020

if YEAR == 2020:
    A74 = [36] # 4th of July is Saturday
    # all-star game at 37 or 30
    as_is_37 = False
    if as_is_37:
        A = [37]
        M = [17, 24, 31, 41, 45, 52]
    else:
        A = [30, 31]
        M = [17, 24, 38, 45, 52]
else:
    A = [37, 38]
    M = [17, 24, 31, 45, 52]
    if YEAR == 2017:
        A74 = [32]
    elif YEAR == 2018:
        A74 = [33]
    elif YEAR == 2019:
        A74 = [34]

M_1 = [i-1 for i in M]
F = WEEKENDS + M_1
F.sort()
print(F)

[0, 1, 7, 8, 14, 15, 16, 21, 22, 23, 28, 29, 35, 37, 42, 43, 44, 49, 50, 51]


In [760]:
nT = 11
nN = 6
nD = 56
nTotalGames = 42
nHalfGames = 21
nSouthGames = 6
nSouthHalf = 3
nNorthIn = 6
nNorthOut = 5
nHalfIn = 3
nHalfOut = 2
nHalfOuts = 3
nSouthNorth = 3
nSNHalf = 1
nSNHalfs = 2

In [761]:
# Sets

# 00#Purcellville 01#Winchester 02#FrontRoyal = North Group 01
# 03#NewMarket 04#Strasburg 05#Woodstock      = North Group 02
# 06#Charlottesville 07#Covington             = South Group
# 08#Harrisonburg 09#Staunton 10#Waynesboro
Team = ["Purcellville", "Winchester", "FrontRoyal", "NewMarket", "Strasburg", "Woodstock", "Charlottesville", "Covington", "Harrisonburg", "Staunton", "Waynesboro"]

# T: all teams in the league
T = np.arange(nT)

# N: teams in the North Division
N = np.arange(0, nN)

# S: teams in the South Division
S = np.arange(nN, nT)

# G1: North Division teams that play more frequently
G1 = np.arange(0, nN//2)

# G2: North Division teams that play more frequently, where G1 ∩ G2  ∅
G2 = np.arange(nN//2, nN)

# D: days during the season
D = np.arange(nD)

# W: weekend (Friday and Saturday) days during the season
W = np.array([0,1, 7,8, 14,15, 21,22, 28,29, 35,36, 42,43, 49,50])

# P: prime days during the season, Fridays and Saturdays and the last three days of the season
P = np.concatenate([W, np.array([53, 54, 55])])

# Su: Sundays during the season
Su = np.array([2, 9, 16, 23, 30, 37, 44, 51])

# F: days during the season that require a full slate of games to be played

# M: strategic idle days across the entire league

# A: days of the All-Star break

# 33: days when Winchester does not want to
#     play on the road against Covington or Charlottesville
i33 = np.array([i for i in T if Team[i] == "Winchester"])
j33 = np.array([i for i in T if Team[i] == "Covington" or Team[i] == "Charlottesville"])
k33 = np.array([25, 26, 27, 28, 39, 40, 41, 42])

# 34: day when Waynesboro travels to play in Charlottesville
i34 = np.array([i for i in T if Team[i] == "Charlottesville"])
j34 = np.array([i for i in T if Team[i] == "Waynesboro"])
k34 = np.array([i for i in A74])
Tof6 = np.array([i34[0], j34[0]])
Tof7 = np.array([i for i in T if i not in Tof6])

# K: days of the dth week of the season, for d  1,:::,|D|/=7
K = np.array([np.arange(7 * i, 7 * (i + 1)) for i in range(nD//7)])

WEEK = np.array([0, 1, 2, 3, 4, 5, 6, 7])
TWO = np.array([0, 1])
THREE = np.array([0, 1, 2])
FOUR = np.array([0, 1, 2, 3])
ELEVEN = np.arange(11)
D_1 = np.arange(0, nD - 1)
D_2 = np.arange(0, nD - 2)
D_10 = np.arange(0, nD - 10)
D_M_A_74 = np.array([i for i in D if i not in A and i not in M and i not in A74])

In [762]:
# Parameters
#gd: minimum number of games each team must play during the d th week of the season, for d  1,:::,|D|=7
Gmin = [6, 6, 4, 4, 3, 3, 4, 4]

# δij: equals 1 if team i and team j are in the same division and 0 otherwise
Same = [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]]

# γik: equals 1 if team i cannot play a home game on day k and 0 otherwise
Yhome = np.zeros((11, 56)) ################ TBD ################

# dij: rank order of travel distance in miles for each team i to play at opponent j,
#      where rank 1 indicates the farthest opponent of team i
if not is_line:
    Dis = np.array([
        [11, 10,  9,  6,  8,  7,  4,  1,  5,  3,  2],
        [ 8, 11,  9,  6, 10,  7,  3,  1,  5,  4,  2],
        [ 7,  9, 11,  6, 10,  8,  4,  1,  5,  2,  3],
        [ 2,  5,  7, 11,  8, 10,  3,  1,  9,  6,  4],
        [ 6,  8, 10,  7, 11,  9,  2,  1,  5,  4,  3],
        [ 5,  7,  8,  9, 10, 11,  2,  1,  6,  4,  3],
        [ 2,  3,  6,  7,  4,  5, 11,  1,  8,  9, 10],
        [ 1,  2,  3,  6,  4,  5,  7, 11,  8, 10,  9],
        [ 1,  3,  4, 10,  6,  7,  5,  2, 11,  9,  8],
        [ 1,  2,  3,  7,  4,  6,  8,  5,  9, 11, 10],
        [ 1,  2,  4,  7,  3,  6,  9,  5,  8, 10, 11]
    ])
elif is_line:
    Dis = np.array([
        [99, 10,  9,  6,  8,  7,  4,  1,  5,  2,  3,],
        [ 8, 99, 10,  6,  9,  7,  4,  1,  5,  3,  2,],
        [ 7,  9, 99,  6, 10,  8,  4,  1,  5,  2,  3,],
        [ 2,  3,  7, 99,  8, 10,  4,  1,  9,  6,  5,],
        [ 6,  8, 10,  7, 99,  9,  4,  1,  5,  3,  2,],
        [ 5,  7,  8,  9, 10, 99,  4,  1,  6,  3,  2,],
        [ 1,  3,  5,  7,  4,  6, 99,  2,  8,  9, 10,],
        [ 1,  2,  3,  6,  4,  5,  7, 99,  8, 10,  9,],
        [ 1,  3,  4, 10,  5,  7,  6,  2, 99,  9,  8,],
        [ 1,  2,  3,  7,  4,  5,  8,  6,  9, 99, 10,],
        [ 1,  2,  3,  7,  4,  6,  9,  5,  8, 10, 99,]
        ])


In [763]:
# Decision variables

# Xijk = 1 if team i is home to team j on day k
X = {
    (i, j, k): xp.var(name=f"X_{i}_{j}_{k}", vartype=xp.binary)
    for i in T for j in T for k in D
}

In [764]:
# Objective

model = xp.problem()

model.addVariable(X)

In [765]:
# Constraints
# A.1
# Objective function X[i, j, k] * Same[i, j]
# A.2
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for j in T for k in D) == nTotalGames for i in T)
# A.3
model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in D) == nHalfGames for i in T)
# A.4
model.addConstraint(xp.Sum(X[i, i, k] for k in D) == 0 for i in T)
# A.5
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for j in T) <= 1 for i in T for k in D)

In [766]:
# A.6
model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in W) >= 7 for i in T)
# model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in W) >= 7 for i in Tof7)
# model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in W) >= 6 for i in Tof6)
# A.7
model.addConstraint(xp.Sum(X[i, j, k] for i in T for j in T) >= 5 for k in F)
# A.8
MA = np.concatenate((M, A))
model.addConstraint(xp.Sum(X[i, j, k] for i in T for j in T for k in MA) == 0)
# A.9
model.addConstraint(xp.Sum(X[i, j, k] for j in T) == 0 for i in T for k in D if Yhome[i, k] == 1)

In [767]:
# A.10
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for k in D) == nSouthGames for i in S for j in S if i != j)
# A.11
model.addConstraint(xp.Sum(X[i, j, k] for k in D) == nSouthHalf for i in S for j in S if i != j)
# A.12
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for k in D) == nNorthIn for i in G1 for j in G1 if i != j)
# A.13
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for k in D) == nNorthIn for i in G2 for j in G2 if i != j)
# A.14
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for k in D) == nNorthOut for i in G2 for j in G1)
# A.15
model.addConstraint(xp.Sum(X[i, j, k] for k in D) == nHalfIn for i in G1 for j in G1 if i != j)
# A.16
model.addConstraint(xp.Sum(X[i, j, k] for k in D) == nHalfIn for i in G2 for j in G2 if i != j)
# A.17
model.addConstraint(xp.Sum(X[i, j, k] for k in D) >= nHalfOut for i in G2 for j in G1)
model.addConstraint(xp.Sum(X[i, j, k] for k in D) <= nHalfOuts for i in G2 for j in G1)
# A.18
model.addConstraint(xp.Sum(X[i, j, k] for k in D) >= nHalfOut for i in G1 for j in G2)
model.addConstraint(xp.Sum(X[i, j, k] for k in D) <= nHalfOuts for i in G1 for j in G2)
# A.19
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for k in D) == nSouthNorth for i in S for j in N)
# A.20
model.addConstraint(xp.Sum(X[i, j, k] for k in D) >= nSNHalf for i in S for j in N)
model.addConstraint(xp.Sum(X[i, j, k] for k in D) <= nSNHalfs for i in S for j in N)

In [768]:
# A.21
model.addConstraint(xp.Sum(X[i, j, z+a] + X[j, i, z+a] for j in T for a in ELEVEN) <= 10 for i in T for z in D_10)
# A.22 ZZZZZZZZZZZZZZZZ????????????????
model.addConstraint(xp.Sum(X[i, j, D_M_A_74[k+a]] + X[j, i, D_M_A_74[k+a]] for j in T for a in TWO) >= 1 for i in T for k in range(len(D_M_A_74)-1))
# A.23
model.addConstraint(xp.Sum(X[i, j, z+a] for j in T for a in THREE) <= 2 for i in T for z in D_2)
# A.24
model.addConstraint(xp.Sum(X[i, j, z+a] for i in T for a in THREE) <= 2 for j in T for z in D_2)
# A.25
model.addConstraint(xp.Sum(X[i, j, z-1] + X[j, i, z-1] + X[i, j, z+1] + X[j, i, z+1] for j in T) >= 1 for i in T for z in M)
# A.26
model.addConstraint(xp.Sum(X[i, j, A[0]-1] + X[j, i, A[0]-1] + X[i, j, A[0]+2] + X[j, i, A[0]+2] for j in T) >= 1 for i in T)

In [769]:
# A.27
model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in K[d]) >= 2 for i in T for d in WEEK)
model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in K[d]) <= 4 for i in T for d in WEEK)
# A.28
model.addConstraint(xp.Sum(X[j, i, k] for j in T for k in K[d]) >= 2 for i in T for d in WEEK)
model.addConstraint(xp.Sum(X[j, i, k] for j in T for k in K[d]) <= 4 for i in T for d in WEEK)
# A.29
model.addConstraint(xp.Sum(X[i, j, k] + X[j, i, k] for j in T for k in K[d]) >= Gmin[d] for i in T for d in WEEK)
# A.30
model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in Su) >= 3 for i in T)
model.addConstraint(xp.Sum(X[i, j, k] for j in T for k in Su) <= 4 for i in T)
# A.31
model.addConstraint(xp.Sum(X[i, j, z+a] + X[j, i, z+a] for a in THREE) <= 1 for i in T for j in T for z in D_2)
# A.32
model.addConstraint(xp.Sum(X[j, i, z+a] for a in TWO for j in T if Dis[i, j] <= 3) <= 1 for i in T for z in D_1)

In [770]:
# A.33
if YEAR == 2020:
    model.addConstraint(xp.Sum(X[j, i, k] for i in i33 for j in j33 for k in k33) == 0)
# A.34
model.addConstraint(xp.Sum(X[i, j, k] for i in i34 for j in j34 for k in k34) == 1)
model.addConstraint(xp.Sum(X[i, j, k] for i in T for j in T for k in k34) == 1)
# A.35
# X[i, j, k] is binary for i in T, j in T, k in D

In [771]:
Team_00 = [  # PurcellvilleCannons 
    [2,8], [2,1], [1,6], [0,-1], [1,5], [2,1], [1,7],
    [2,5], [1,3], [2,10], [1,5], [0,-1], [2,3], [1,5],
    [1,4], [2,2], [2,6], [0,-1], [2,10], [1,8], [0,-1],
    [1,2], [2,1], [1,10], [0,-1], [2,2], [1,3], [0,-1]
]
Team_01 = [  # WinchesterRoyals 
    [1,5], [1,0], [2,9], [2,7], [1,2], [1,0], [0,-1],
    [2,3], [1,5], [2,7], [1,10], [1,3], [2,6], [1,8],
    [1,2], [2,5], [0,-1], [0,-1], [2,8], [2,5], [1,7],
    [2,4], [1,0], [1,9], [0,-1], [0,-1], [2,8], [0,-1]
]
Team_02 = [  # FrontRoyalCardinals  
    [1,4], [2,5], [0,-1], [1,3], [2,1], [1,10], [2,6],
    [1,4], [2,8], [1,6], [2,4], [2,7], [0,-1], [1,10],
    [2,1], [1,0], [1,8], [0,-1], [2,4], [2,9], [0,-1],
    [2,0], [1,5], [2,3], [0,-1], [1,0], [1,9], [0,-1]
]
Team_03 = [  # NewMarketRebels 
    [0,-1], [2,4], [1,10], [2,2], [1,6], [1,9], [2,4],
    [1,1], [2,0], [0,-1], [2,9], [2,1], [1,0], [1,7],
    [2,5], [2,4], [1,7], [0,-1], [1,5], [0,-1], [2,8],
    [2,6], [1,4], [1,2], [0,-1], [2,9], [2,0], [0,-1]
]
Team_04 = [  # StrasburgExpress 
    [2,2], [1,3], [2,5], [0,-1], [2,9], [1,6], [1,3],
    [2,2], [0,-1], [2,8], [1,2], [2,5], [1,9], [2,6],
    [2,0], [1,3], [1,9], [0,-1], [1,2], [2,7], [0,-1],
    [1,1], [2,3], [1,6], [0,-1], [2,5], [1,10], [0,-1]
]
Team_05 = [  # WoodstockRvrBndts 
    [2,1], [1,2], [1,4], [2,8], [2,0], [1,7], [0,-1],
    [1,0], [2,1], [1,9], [2,0], [1,4], [2,10], [2,0],
    [1,3], [1,1], [2,10], [0,-1], [2,3], [1,1], [1,6],
    [0,-1], [2,2], [1,7], [0,-1], [1,4], [1,6], [2,7]
]
Team_06 = [  # CharlottesvilleTomSox 
    [2,7], [0,-1], [2,0], [1,10], [2,3], [2,4], [1,2],
    [2,9], [1,7], [2,2], [1,8], [0,-1], [1,1], [1,4],
    [2,7], [1,9], [1,0], [0,-1], [1,7], [2,10], [2,5],
    [1,3], [2,9], [2,4], [0,-1], [1,8], [2,5], [1,10]
]
Team_07 = [  # CovingtonLumberjacks 
    [1,6], [1,9], [2,8], [1,1], [0,-1], [2,5], [2,0],
    [1,10], [2,6], [1,1], [0,-1], [1,2], [2,8], [2,3],
    [1,6], [1,8], [2,3], [0,-1], [2,6], [1,4], [2,1],
    [2,10], [1,8], [2,5], [0,-1], [2,10], [0,-1], [1,5]
]
Team_08 = [  # HarrisonburgTurks 
    [1,0], [2,10], [1,7], [1,5], [2,10], [0,-1], [2,9],
    [0,-1], [1,2], [1,4], [2,6], [1,9], [1,7], [2,1],
    [0,-1], [2,7], [2,2], [0,-1], [1,1], [2,0], [1,3],
    [1,9], [2,7], [0,-1], [0,-1], [2,6], [1,1], [0,-1]
]
Team_09 = [  # StauntonBraves 
    [1,10], [2,7], [1,1], [0,-1], [1,4], [2,3], [1,8],
    [1,6], [2,10], [2,5], [1,3], [2,8], [2,4], [0,-1],
    [1,10], [2,6], [2,4], [0,-1], [0,-1], [1,2], [1,10],
    [2,8], [1,6], [2,1], [0,-1], [1,3], [2,2], [0,-1]
]
Team_10 = [  # WaynesboroGenerals 
    [2,9], [1,8], [2,3], [2,6], [1,8], [2,2], [0,-1],
    [2,7], [1,9], [1,0], [2,1], [0,-1], [1,5], [2,2],
    [2,9], [0,-1], [1,5], [0,-1], [1,0], [1,6], [2,9],
    [1,7], [0,-1], [2,0], [0,-1], [1,7], [2,4], [2,6]
]

In [772]:
# 顺序使用Team_00到Team_10
if is_rep:
    for i in range(nT):
        if i == 0:
            Team_i = Team_00
        elif i == 1:
            Team_i = Team_01
        elif i == 2:
            Team_i = Team_02
        elif i == 3:
            Team_i = Team_03
        elif i == 4:
            Team_i = Team_04
        elif i == 5:
            Team_i = Team_05
        elif i == 6:
            Team_i = Team_06
        elif i == 7:
            Team_i = Team_07
        elif i == 8:
            Team_i = Team_08
        elif i == 9:
            Team_i = Team_09
        elif i == 10:
            Team_i = Team_10
        for ii in range(len(Team_i)):
            if Team_i[ii][0] == 0:
                model.addConstraint(xp.Sum(X[i, j, ii]+X[j,i,ii] for j in T) == 0)
            elif Team_i[ii][0] == 1:
                model.addConstraint(X[i, Team_i[ii][1], ii] == 1)
            elif Team_i[ii][0] == 2:
                model.addConstraint(X[Team_i[ii][1], i, ii] == 1)

In [ ]:
# 00#Purcellville 01#Winchester 02#FrontRoyal =
# 03#NewMarket 04#Strasburg 05#Woodstock      =
# 06#Charlottesville 07#Covington             =
# 08#Harrisonburg 09#Staunton 10#Waynesboro
Pop = np.array([0.687714, 0.775660, 0.605491, 0.795704, 0.587514, 0.488740,
                0.949316, 0.482266, 0.832510, 0.719611, 0.756830])

MMAX = xp.Sum((X[i, j, k] * Same[i][j] * (Pop[i] + Pop[j])) for i in T for j in T for k in P)

In [ ]:
# Objective function
MMAX = xp.Sum((X[i, j, k] * Same[i][j]) for i in T for j in T for k in P)

model.setObjective(MMAX, sense=xp.maximize)
model.setControl("MAXTIME", 120)
model.setControl("MIPDUALREDUCTIONS", 0)   # 禁用对偶缩减，避免遗漏可行解
model.setControl("MAXMIPSOL", 1000)         # 最多找到1000个可行解

model.solve()

FICO Xpress v9.4.2, Hyper, solve started 16:01:45, Mar 27, 2025
Heap usage: 12MB (peak 12MB, 284KB system)
Maximizing MILP noname using up to 16 threads and up to 27GB memory, with these control settings:
MAXTIME = 120
MAXMIPSOL = 1000
OUTPUTLOG = 1
MIPDUALREDUCTIONS = 0
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
     10761 rows         6776 cols       317329 elements      6776 entities
Presolved problem has:
      5285 rows         5258 cols       179477 elements      5258 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 14MB (peak 26MB, 284KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  2.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  4.20e+01] / [ 1.00e+00,  1.50e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
Autoscaling applied standard scaling

Will try to keep branch and

In [ ]:
num_solutions = model.getAttrib("MIPSOLS")
print(f"找到的可行解总数: {num_solutions}")    

找到的可行解总数: 1


In [ ]:
alll = 0
# pp 是一个2行11列的矩阵
pp = np.zeros((2, 11))
for i in T:
    for j in T:
        for k in P:
            if model.getSolution(X[i, j, k]) > 0.5:
                alll += 1
                pp[0][i] += 1
                if Same[i][j] == 1:
                    pp[0][i] = pp[0][i] + 0.01
                pp[1][i] += 1
                pp[1][j] += 1
print(alll)
print(model.getObjVal()/alll)
print(pp)

91
0.978021978021978
[[ 9.09  8.08  9.09  8.08  8.08 10.1   6.06  8.07  9.08  9.09  7.07]
 [17.   18.   17.   18.   18.   18.   14.   15.   15.   15.   17.  ]]


Percentage of Prime Day Games Scheduled to Hold Intradivision Games
在原文中：比例是 88/91 = 96.70%
本次复现：比例是 89/91 = 97.80%

In [ ]:
print(YEAR)
print(f"Total Games: {model.getObjVal()}")
print()

for ele in range(nT):
    for eight in range(8):
        for sev in range(7):
            have = False
            for ele2 in range(nT):
                if model.getSolution(X[ele, ele2, sev+eight*7]) > 0.5:
                    have = True
                    if ele2 < 10:
                        print(ele2, end="")
                    elif ele2 == 10:
                        print("A", end="")
                if model.getSolution(X[ele2, ele, sev+eight*7]) > 0.5:
                    have = True
                    print("=", end="")
            if not have:
                print('!', end="")
        print("  ", end="")
    print()

2020
Total Games: 89.0

3==!2==  1=!2=49  ==!!=91  =3A!=!=  5!!!2==  3!=!67=  4=A!==!  1=8!=54  
=0A=9!5  ==7!4=8  0==!!3=  =56!=!9  =3!!A!=  =!=!82=  54=!0!=  =2=!=2=  
1==8=4!  =36=9=7  =58!==!  04=!1==  1=!!=!5  =!!!5=0  !=7!=!A  4==!0=3  
==6==0!  ==8=5=!  21=!2==  5=!!4A=  4=!!6=9  =!4!9=1  =2!!=7!  50!!1==  
=27=!==  31!9==A  =03!!=6  1==!=8!  ==!!50!  2!=!!=7  ==5!=61  =5A!=3=  
43!96==  20=8=!=  4==!6!A  ==8!=!3  =2!!=7=  1!0!=!=  =0=!3=!  ==7!4=1  
=7=4=!0  9!=A0=5  =!1!=7=  A==!!27  =8!!==1  !A=!=38  ==9!==8  !92!==!  
A==13!=  !==3=1=  =A5!==8  !=2!09=  =4!!9=0  8!6!!==  96=!6=!  8==!A!=  
6=0=A=!  A7==73=  !9=!04=  9!=!==2  7=!!!6=  =!9!=4=  3=1!!5=  =!=!6=A  
!82==84  =A!==6=  7==!7=3  =64!5==  6A!!=!=  A!=!==5  =!=!20!  ==1!!87  
=!=3=57  ==5=!2=  6=9!!2=  =7=!8=0  !=!!=38  ==1!!9!  68=!4!=  97=!=6=  


In [ ]:
print(f"Total Games: {model.getObjVal()}")
print()

# # 假设 nD 表示赛季总天数，T 表示球队列表，X 为决策变量字典
# # 按周输出赛程，每周7天
# for week in range(nD // 7):
#     print(f"Week {week+1}")
#     # 打印表头
#     header = "Team".ljust(15) + " | " + " ".join([f"Day{k+1}" for k in range(7)])
#     print(header)
#     print("-" * len(header))
    
#     for i in T:
#         schedule = []
#         # 遍历一周的7天，day_index 为全赛季内的具体日期索引
#         for k in range(7):
#             day_index = week * 7 + k
#             # 检查球队 i 当天是否有比赛（无论是主场还是客场）
#             game_found = False
#             for j in T:
#                 if model.getSolution(X[i, j, day_index]) > 0.5 or model.getSolution(X[j, i, day_index]) > 0.5:
#                     game_found = True
#                     break
#             if game_found:
#                 schedule.append("1")
#             else:
#                 schedule.append(".")
#         # 将球队编号转换为字符串后再调用 ljust
#         print(f"{str(i).ljust(15)} | " + " ".join(schedule))
#     print()

# 统计总比赛场次
TT = 0
for week in range(nD // 7):
    for i in T:
        for k in range(7):
            for j in T:
                if model.getSolution(X[i, j, k+7*week]) > 0.5:
                    TT += 1
print("Total number of games (sum of 1's):", TT)


Total Games: 89.0

Total number of games (sum of 1's): 231
